In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import csv

# 1 способ:

In [4]:
train = pd.read_csv('train.csv', names=['text', 'label'])
train.head()

,text,label
0,text,label
1,ENT магнит,0
2,Сообщение от подписчиков Поступило сообщение о...,-1
3,Курт Воннегут « Колыбель для кошки » Продолжаю...,1
4,Надо поугарать взять пакет из магнита и закупи...,0


In [21]:
test = pd.read_csv('test.csv', names=['text', 'label'])
test.head()

,text,label
0,text,label
1,в Новый год ОРХИДЕЯ пойдёт с чистой ПОП* о...,0
2,Какой у вас денежный сценарий Если у бизнесмен...,0
3,Москвича избили и порезали при попытке купить ...,-1
4,Свинина тушеная Йошкар-Олинский мясокомбинат `...,-1


# Baseline

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [23]:
X_train = train.text
y_train = train.label
X_test = test.text
y_test = test.label

In [24]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB())
              ])

In [25]:
nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [26]:
%%time 
y_pred = nb.predict(X_test)

Wall time: 5.49 s


In [27]:
print('Accuracy: {:.2f}'.format(accuracy_score(y_pred, y_test)))

Accuracy: 0.63


In [28]:
submission_1 = test.copy(deep=True)

In [29]:
submission_1.lable = nb.predict(test.text)

C:\Users\komle\.conda\envs\tf1\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [30]:
submission_1

,text,label
0,text,label
1,в Новый год ОРХИДЕЯ пойдёт с чистой ПОП* о...,0
2,Какой у вас денежный сценарий Если у бизнесмен...,0
3,Москвича избили и порезали при попытке купить ...,-1
4,Свинина тушеная Йошкар-Олинский мясокомбинат `...,-1
...,...,...
9962,Мука пшеничная экстра `` Французская штучка ''...,1
9963,Не знаю почему именно в ENT такой бардак Почем...,-1
9964,масло по этой же цене в магазине Сказка брала ...,1
9965,В Самаре гипермаркет ENT хочет добиться « разв...,-1


In [31]:
submission_1.to_csv('submission_1_nb.csv', sep=',', index=False)

# 2 способ:

Здесь я сделала стандатную для NLP предобработку данных. От этого точность изменилась не сильно 

In [32]:
train = pd.read_csv('train.csv', names=['text', 'label'])
train.head()

test = pd.read_csv('test.csv', names=['text', 'label'])
test.head()

,text,label
0,text,label
1,в Новый год ОРХИДЕЯ пойдёт с чистой ПОП* о...,0
2,Какой у вас денежный сценарий Если у бизнесмен...,0
3,Москвича избили и порезали при попытке купить ...,-1
4,Свинина тушеная Йошкар-Олинский мясокомбинат `...,-1


In [33]:
import pymorphy2, re

morph = pymorphy2.MorphAnalyzer()

def clean_text(text):
    new_text = ''
    grams_exclusion = {'PREP', 'CONJ', 'INTJ', 'Name', 'Surn', 'Patr', 'Orgn', 'Trad'}
    text = re.sub('[^а-яА-Яa-zA-ZёЁ]', ' ', text)

    words = text.split()
    for word in words:
        p = morph.parse(word)[0]
        if not any(tag in p.tag for tag in grams_exclusion):
            new_text += ' ' + p.normal_form
    new_text = new_text[1:]
    return new_text

train['text_lem'] = train['text'].apply(lambda x: clean_text(x))
test['text_lem'] = test['text'].apply(lambda x: clean_text(x))

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [35]:
X_train = train.text_lem
y_train = train.label
X_test = test.text_lem
y_test = test.label

In [36]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB())
              ])

In [37]:
nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [39]:
%%time 
y_pred = nb.predict(X_test)

Wall time: 3.46 s


In [40]:
print('Accuracy: {:.2f}'.format(accuracy_score(y_pred, y_test)))

Accuracy: 0.63


In [38]:
submission_2 = test.copy(deep=True)
submission_2.lable = nb.predict(test.text)
submission_2
submission_2.to_csv('submission_2_nb.csv', sep=',', index=False)

C:\Users\komle\.conda\envs\tf1\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


# 3 способ:

К сожалению, точность все еще оставляет желать лучшего и я вспомнила о том что в нашем корпусе отмечены сущности и нашла статью от Стенфорда:http://cs224d.stanford.edu/reports/AhresY.pdf. К сожалению, времени на ее реализацию у меня не осталось 